In [14]:
import pandas as pd
train = pd.read_csv('../dataset/train.csv', index_col=0) # 学習用データ
test = pd.read_csv('../dataset/test.csv', index_col=0) # 学習用データ   # 評価用データ
sample_submit = pd.read_csv('../dataset/sample_submission.csv', index_col=0, header=None) # 応募用サンプルファイル
train.head(5)

,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural
0,163,21,1.0,0,0,1,N,N,31-Jan-98,1,0,22-Sep-06,2006,PHOENIX,AZ,SD,"$80,000.00","$80,000.00","$68,000.00",0
1,84,6,1.0,4,0,0,0,N,31-Oct-93,1,62,30-Jun-92,1992,MCALESTER,OK,OK,"$287,000.00","$287,000.00","$229,600.00",0
2,242,45,1.0,4,90,0,N,N,31-Aug-01,1,42,18-Apr-01,2001,HAWTHORNE,NJ,NJ,"$31,983.00","$30,000.00","$15,000.00",1
3,237,4,1.0,0,0,0,N,N,31-Aug-07,1,33,6-Oct-03,2004,NASHVILLE,TN,SD,"$229,000.00","$229,000.00","$229,000.00",0
4,184,0,1.0,0,0,0,N,N,8-Jun-83,1,0,17-Dec-99,2000,POMONA,CA,CA,"$525,000.00","$525,000.00","$393,750.00",0


In [16]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


numeric_columns =['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'RevLineCr', 'LowDoc',
       'MIS_Status', 'Sector', 'ApprovalFY', 'State',
       'BankState', 'DisbursementGross', 'GrAppv', 'SBA_Appv', 'UrbanRural']

test_numeric_columns =['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'RevLineCr', 'LowDoc','Sector', 'ApprovalFY', 'State',
       'BankState', 'DisbursementGross', 'GrAppv', 'SBA_Appv', 'UrbanRural']
train_numeric = train[numeric_columns]
test_numeric = test[test_numeric_columns]
# Convert categorical columns to one-hot encoded columns
categorical_columns = ['RevLineCr', 'LowDoc', 'UrbanRural']
train_numeric = pd.get_dummies(train, columns=categorical_columns)
test_numeric = pd.get_dummies(test, columns=categorical_columns)


# Convert currency columns to numeric
train_numeric['DisbursementGross'] = pd.to_numeric(train_numeric['DisbursementGross'], errors='coerce')
test_numeric['DisbursementGross'] = pd.to_numeric(test_numeric['DisbursementGross'], errors='coerce')

train_numeric['GrAppv'] = pd.to_numeric(train_numeric['GrAppv'], errors='coerce')
test_numeric['GrAppv'] = pd.to_numeric(test_numeric['GrAppv'], errors='coerce')

train_numeric['SBA_Appv'] = pd.to_numeric(train_numeric['SBA_Appv'], errors='coerce')
test_numeric['SBA_Appv'] = pd.to_numeric(test_numeric['SBA_Appv'], errors='coerce')

# Convert state columns to score of GDP
state_data = pd.read_csv('../dataset/state_gdp.csv', index_col=0)
state_columns = ['State', 'BankState']
for col in state_columns:
    train_numeric[col] = train_numeric[col].map(state_data.set_index('State')['gdp'])
    test_numeric[col] = test_numeric[col].map(state_data.set_index('State')['gdp'])



# 数値列だけを選択
numeric_columns = train_numeric.select_dtypes(include=['number']).columns
# 数値列だけを対象に平均計算
train_numeric[numeric_columns] = train_numeric[numeric_columns].fillna(train_numeric[numeric_columns].mean())
test_numeric[test_numeric_columns] = test_numeric[test_numeric_columns].fillna(test_numeric[test_numeric_columns].mean())
# 残りの文字列列に対しては、NaN をそのままにするか、適切な処理を行う
# 例えば、文字列列に対しては欠損値を特定の文字列で置き換えたりすることが考えられます

X = train_numeric.drop(['DisbursementDate', 'ApprovalDate', 'City','MIS_Status'], axis=1)
y = train_numeric['MIS_Status']                # 目的変数

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.dtypes)

# LightGBMモデルの設定
lgbm = lgb.LGBMClassifier()

# モデルの訓練
lgbm.fit(X_train, y_train)

# モデルの評価
y_pred_categorical = lgbm.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred_categorical)}')


KeyError: "['RevLineCr', 'LowDoc', 'UrbanRural'] not in index"

In [ ]:
from sklearn.metrics import f1_score
  
mean_f1 = f1_score(y_test, y_pred_categorical, average='macro')
print(f'Mean F1 Score: {mean_f1}')

Mean F1 Score: 0.6344638947781297


In [ ]:
predictions = lgbm.predict(test[numeric_columns.drop('MIS_Status')] )
# 結果を表示
print(predictions)
sample_submit[1] = predictions
sample_submit.to_csv('submit.csv', header=None)

ValueError: Number of features of the model must match the input. Model n_features_ is 26 and input n_features is 13